**Dependencies and Constants**

In [18]:
import os
import cv2
from imutils import paths
from bs4 import BeautifulSoup as bs

BASE_PATH = "Data/AAU_RainSnow"
EGENSEVEJ_BASE = "os.path.join(BASE_PATH, 'Egensevej')"
EGENSEVEJ_1 = os.path.join(EGENSEVEJ_BASE, 'Egensevej-1')

**Configuration**

In [14]:
# selective search 
MAX_PROPOSALS = 2000
MAX_PROPOSALS_INFER = 200

MAX_POSITIVE_IMAGES = 30
MAX_NEGATIVE_IMAGES = 10

INPUT_DIMENSIONS = (256, 256)

MODEL_PATH = "classifier.h5"
ENCODER_PATH = "label_encoder.pickle"

MIN_PROBABILITY = 0.95

**Define Intersection Over Union Method**

In [15]:
def compute_iou(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the intersection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

**Loop over images**

In [19]:
imagePaths = paths.list_images(EGENSEVEJ_1)

In [20]:
# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
	# show a progress report
	print("[INFO] processing image {}/{}...".format(i + 1,
		len(imagePaths)))
	# extract the filename from the file path and use it to derive
	# the path to the XML annotation file
	filename = imagePath.split(os.path.sep)[-1]
	filename = filename[:filename.rfind(".")]
	annotPath = os.path.sep.join([config.ORIG_ANNOTS,
		"{}.xml".format(filename)])
	# load the annotation file, build the soup, and initialize our
	# list of ground-truth bounding boxes
	contents = open(annotPath).read()
	soup = bs(contents, "html.parser")
	gtBoxes = []
	# extract the image dimensions
	w = int(soup.find("width").string)
	h = int(soup.find("height").string)